# Hi!

This notebook was created using a local Apache Spark cluster.

If you are curious on what the deployment looks like,
see [this](https://github.com/kamadorueda/machine/blob/main/nixos-modules/spark/default.nix).

The cell below is the only thing that needs to be changed if you want to run it,
either on DataBricks or TALC.

Thank you!


In [12]:
import pyspark

# Step 1: Please change `sc` and `spark`
# so that they point to your spark cluster (TALC, DataBricks, etc)

conf = pyspark.SparkConf().setMaster("local[4]")
sc = pyspark.SparkContext.getOrCreate(conf)
spark = pyspark.sql.SparkSession.builder.config(conf=conf).getOrCreate()

# Step 2: Please set the path to the files
ISSUES_DIR = "../step-1-structured-data/issues"
LABELS = "../step-2-labeling/labels-filled.csv"

In [13]:
import csv

with open(LABELS) as file:
    labels = tuple(csv.DictReader(file))

labels[0]

{'issue-number': '18544', 'was-fixed': 'yes'}

In [15]:
# issue_numbers

import json
import os.path

data = []

for label in labels:
    issue_number = label["issue-number"]
    assert label["was-fixed"] in ("yes", "no")

    with open(os.path.join(ISSUES_DIR, f"{issue_number}.json")) as file:
        issue = json.load(file)

    data.append(
        {
            "issue_number": issue_number,
            "title": issue["title"],
            "body": issue["body"],
            "was_fixed": label["was-fixed"] == "yes",
        }
    )


data = spark.createDataFrame(data)

data.show()


+--------------------+------------+--------------------+---------+
|                body|issue_number|               title|was_fixed|
+--------------------+------------+--------------------+---------+
|### Prerequisites...|       18544|PowerShell 7.3.0 ...|     true|
|### Prerequisites...|       18540|Handling of QUOTA...|     true|
|### Prerequisites...|       18501|Problem with PSNa...|    false|
|### Summary of th...|       18500|Questions.. and o...|     true|
|### Summary of th...|       18497|Define env vars i...|     true|
|### Prerequisites...|       18490|Write-Error prefi...|    false|
|### Prerequisites...|       18489|SemanticVersion d...|     true|
|### Prerequisites...|       18485|ErrorResponseExce...|     true|
|### Prerequisites...|       18478|Please remove ref...|     true|
|### Prerequisites...|       18476|Class not registe...|     true|
|### Prerequisites...|       18463|`Get-Help -Parame...|     true|
|### Summary of th...|       18460| `Invoke-PowerShell`|     t